In [13]:
!pip install requests
import requests


In [ ]:
def getTxnsForHash(hash):
    idx =0
    responseList =[]
    payload={}
    headers = {}
    while True:
        try:
            url = F"https://blockstream.info/api/block/{hash}/txs/{idx*25}"
            idx+=1
            response = requests.request("GET", url, headers=headers, data=payload)
            for  txn in response.json():
                responseList.append(txn)
        except:
            break
    return responseList

# getTxnsForHash("000000000000000000076c036ff5119e5a5a74df77abf64203473364509f7732")

In [15]:
def getBlockTransactionHash(block):
    url = F"https://blockstream.info/api/block-height/{str(block)}"
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    return response.text


https://blockstream.info/api/block-height/680000


'000000000000000000076c036ff5119e5a5a74df77abf64203473364509f7732'

In [16]:
def setCurrentBlockTxns(transactionList , transactionMap):
    for idx in range(1, len(transactionList)):
        transactionMap[transactionList[idx]['txid']] = True

In [38]:
def getCountParenttransactions(transactionList , transtionInBlock,parents,parentTransactionCountMap):
    for idx in range (1, len(transactionList)):
        currentTxn = transactionList[idx]['txid']
        parentTxns= transactionList[idx]['vin']
        for i in range(len(parentTxns)):
            if transtionInBlock.get(parentTxns[i]['txid'], False):
                parentTransactionCountMap[currentTxn] = parentTransactionCountMap.get(currentTxn ,0)+1
                if currentTxn not in parents:
                    parents[currentTxn] = []
                parents[currentTxn].append(parentTxns[i]['txid'])


In [40]:
def getTotalCount(parentTxnCount, parentMap):
    for key in parentTxnCount:
        parents = parentMap[key]
        for txid in parents:
            parentTxnCount[key] = parentTxnCount.get(key, 0) + parentTxnCount.get(txid, 0)

In [44]:
def transactionAncestorySets(blockNumber):
    transactionHash= getBlockTransactionHash(blockNumber)
    transactionList = getTxnsForHash(transactionHash)
    transcationInCurrentBlock={}
    setCurrentBlockTxns(transactionList, transcationInCurrentBlock)
    parentsMap ={}
    parentTxnCountMap ={}
    getCountParenttransactions(transactionList,transcationInCurrentBlock,parentsMap,parentTxnCountMap)
    getTotalCount(parentTxnCountMap,parentsMap)
    sortedCount = sorted(parentTxnCountMap.items(), key=lambda kv:
                 (kv[1], kv[0]),reverse=True)
    for idx in range(10):
        print(sortedCount[idx][1] , sortedCount[idx][0])
    # return sortedCount[:10]

transactionAncestorySets(680000)

https://blockstream.info/api/block-height/680000
14 7d08f0c61cda9379bdf1fa68095f827199a0d4cb6b466a6da3f0dc956772c52b
13 b2bab595112517e8b6a06aa9f616272b479e57e21b4da52877ddf385316aa19b
12 d294be35db0b5fab4a6a00d6e4441c7e54be88fa02dfc188b75e4604ec6c3fcf
12 7a128b0242d89d327fc2c273199c7529a31477d8ea949e5176b2a4eb69b74464
12 7841dc7cf61d394094f4341fa98d0a6fd771e95ac93a9dcfec12a23ed3c670c5
12 4205c68c68266259c5723948e0407dff25600e6420659cef4286dd1ae4658b63
11 ef6c8e97b62eced1913df503667d49b9f5890cdb201be5d5d6c304af1d3f5db1
11 afe4b90e667df0171f63e6cc95c0a12d24592d436dd2e8b9b2a9998b4099ff6d
11 973e5adb05cc1cb80cabc5e451200333c993034153a078733ec06af7bf3c860b
11 4b4c90943c1651eabb8c5dfb6f490c4e56bd6cf42950a0430db17b9691b0236c


[('7d08f0c61cda9379bdf1fa68095f827199a0d4cb6b466a6da3f0dc956772c52b', 14),
 ('b2bab595112517e8b6a06aa9f616272b479e57e21b4da52877ddf385316aa19b', 13),
 ('d294be35db0b5fab4a6a00d6e4441c7e54be88fa02dfc188b75e4604ec6c3fcf', 12),
 ('7a128b0242d89d327fc2c273199c7529a31477d8ea949e5176b2a4eb69b74464', 12),
 ('7841dc7cf61d394094f4341fa98d0a6fd771e95ac93a9dcfec12a23ed3c670c5', 12),
 ('4205c68c68266259c5723948e0407dff25600e6420659cef4286dd1ae4658b63', 12),
 ('ef6c8e97b62eced1913df503667d49b9f5890cdb201be5d5d6c304af1d3f5db1', 11),
 ('afe4b90e667df0171f63e6cc95c0a12d24592d436dd2e8b9b2a9998b4099ff6d', 11),
 ('973e5adb05cc1cb80cabc5e451200333c993034153a078733ec06af7bf3c860b', 11),
 ('4b4c90943c1651eabb8c5dfb6f490c4e56bd6cf42950a0430db17b9691b0236c', 11)]